In [13]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')

In [15]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, LeaveOneOut
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

In [6]:
numeros = pd.read_csv("classificacao_Q4.csv")

In [7]:
numeros.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x91,x92,x93,x94,x95,x96,x97,x98,x99,target
0,1.696199,-0.792598,-0.349427,-0.464560,3.187014,0.035976,1.033274,-1.504968,0.204693,1.691204,...,1.488142,-0.686337,2.084970,-0.685140,-2.049451,2.015426,1.158477,-0.309441,-1.549833,4.0
1,-0.236696,-2.202342,0.024023,1.497700,-0.069758,-2.467088,1.126529,-0.570557,2.079251,-1.882632,...,0.405567,0.509564,1.374071,-0.016943,-0.429280,-0.895016,1.259566,-0.354139,0.806797,5.0
2,-0.436683,1.563816,-0.895999,-0.580425,0.311060,-0.187369,0.805249,-2.399522,-0.578818,1.586981,...,0.933578,-1.285978,0.503162,0.204829,-0.753835,0.290033,1.721487,1.304518,0.478903,3.0
3,1.425908,0.400055,-0.305038,-0.930251,-2.214549,1.763379,-0.239868,-2.058891,-1.006533,-2.156839,...,-0.849927,1.402768,0.393653,-1.466818,0.152257,-4.004950,0.676342,-1.927319,1.959032,8.0
4,-0.186156,-0.975764,0.594660,-1.181980,-1.443414,-0.797651,-1.252608,-0.060452,0.130702,-2.343517,...,-1.444435,-1.818126,0.446574,0.239328,0.802939,-2.035289,-1.433793,-0.218596,0.619317,9.0


In [8]:
numeros_x = numeros.iloc[:,:-1]
print(numeros_x.head())
print(numeros_x.shape)


numeros_y = numeros.iloc[:,-1:]
print(numeros_y.head())
print(numeros_y.shape)

         x0        x1        x2        x3        x4        x5        x6  \
0  1.696199 -0.792598 -0.349427 -0.464560  3.187014  0.035976  1.033274   
1 -0.236696 -2.202342  0.024023  1.497700 -0.069758 -2.467088  1.126529   
2 -0.436683  1.563816 -0.895999 -0.580425  0.311060 -0.187369  0.805249   
3  1.425908  0.400055 -0.305038 -0.930251 -2.214549  1.763379 -0.239868   
4 -0.186156 -0.975764  0.594660 -1.181980 -1.443414 -0.797651 -1.252608   

         x7        x8        x9    ...          x90       x91       x92  \
0 -1.504968  0.204693  1.691204    ...    -0.534360  1.488142 -0.686337   
1 -0.570557  2.079251 -1.882632    ...     0.498207  0.405567  0.509564   
2 -2.399522 -0.578818  1.586981    ...    -2.630024  0.933578 -1.285978   
3 -2.058891 -1.006533 -2.156839    ...    -0.260665 -0.849927  1.402768   
4 -0.060452  0.130702 -2.343517    ...     1.098563 -1.444435 -1.818126   

        x93       x94       x95       x96       x97       x98       x99  
0  2.084970 -0.685140 -2

In [10]:
def cross_validate_mlp(cv,data_x,data_y):
    scores_train= []
    scores_test= []
    acc_train = []
    acc_test = []
    #auc_train = []
    #auc_test = []
    precision_test = []
    precision_train = []
    recall_test = []
    recall_train = []
    f1_test = []
    f1_train = []
    y_pred_train = []
    y_pred_test = []
    
    for train_idx, test_idx in cv.split(data_x):
        X_train, X_test = data_x.iloc[train_idx], data_x.iloc[test_idx] 
        y_train, y_test = data_y.iloc[train_idx], data_y.iloc[test_idx]
        #print(test_idx)
        model = MLPClassifier(hidden_layer_sizes=(100, ), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', 
                           learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, 
                           random_state=0, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, 
                           nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, 
                           beta_2=0.999, epsilon=1e-08)
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        precision_test.append(precision_score(y_test, y_pred_test,average='micro'))
        precision_train.append(precision_score(y_train, y_pred_train,average='micro'))
        recall_test.append(recall_score(y_test, y_pred_test,average='micro'))
        recall_train.append(recall_score(y_train, y_pred_train,average='micro'))
        f1_test.append(f1_score(y_test, y_pred_test,average='micro'))
        f1_train.append(f1_score(y_train, y_pred_train,average='micro'))
        acc_test.append(accuracy_score(y_test, y_pred_test))
        acc_train.append(accuracy_score(y_train, y_pred_train))
        
        #auc_train.append(roc_auc_score(y_train, y_pred_train))
        #auc_test.append(roc_auc_score(y_test, y_pred_test))
        
    print("TRAIN :")
    print('Accuracy Mean', np.mean(acc_train))
    print('Precision Mean', np.mean(precision_train))
    print('Recall Mean', np.mean(recall_train))
    print('F1 Mean', np.mean(f1_train))
    
    print("TEST :")
    #print('Accuracy List Train: ', acc_test)
    print('Accuracy Mean', np.mean(acc_test))
    print('Precision Mean', np.mean(precision_test))
    print('Recall Mean', np.mean(recall_test))
    print('F1 Mean', np.mean(f1_test))


In [16]:
kf = KFold(n_splits=10,shuffle=False)

model = MLPClassifier(hidden_layer_sizes=(100, ), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', 
                           learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, 
                           random_state=0, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, 
                           nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, 
                           beta_2=0.999, epsilon=1e-08)

cross_validate_mlp(kf, numeros_x, numeros_y)

cval = cross_val_score(model, numeros_x, numeros_y, cv=kf,scoring='accuracy')
print("Mean cval_score (Test): ", np.mean(cval))

cvalidate = cross_validate(model,numeros_x,numeros_y,cv=kf,scoring='accuracy', return_train_score=True)
print(sorted(cvalidate.keys()))
print("Train: ",np.mean(cvalidate['train_score']))
print("Test: ",np.mean(cvalidate['test_score']))

TRAIN :
Accuracy Mean 1.0
Precision Mean 1.0
Recall Mean 1.0
F1 Mean 1.0
TEST :
Accuracy Mean 0.6266666666666667
Precision Mean 0.6266666666666667
Recall Mean 0.6266666666666667
F1 Mean 0.6266666666666667
Mean cval_score (Test):  0.6266666666666667
['fit_time', 'score_time', 'test_score', 'train_score']
Train:  1.0
Test:  0.6266666666666667
